In [3]:
import os
import sys

current_path = os.getcwd()
current_path = os.path.join(current_path, 'Dataset')
print(current_path)
folder_com = os.path.join(current_path, 'Commercial Aero')
folder_uss = os.path.join(current_path, 'USS')

def get_file_as_id(folder_path):
    if not os.path.exists(folder_path):
        return ValueError
    folder_name = folder_path.split('/')[-1]
    id = folder_name.split('.')[0]
    # id = id.split('_')
    # id = ' '.join(id)

    return id

com_file_id =[]
uss_file_id = []

for file in os.listdir(folder_com):
    id = get_file_as_id(os.path.join(folder_com, file))
    com_file_id.append(id)
    print(id)

for file in os.listdir(folder_uss):
    id = get_file_as_id(os.path.join(folder_uss, file))
    uss_file_id.append(id)
    print(id)
    

/home/aravindnair/AI-Agents-For-ST/Dataset
CommAero_AAM_AerSaleCorp_2022
USS_Satcom_Comtech_2021


In [4]:
from dsrag.knowledge_base import KnowledgeBase
from dsrag.llm import OpenAIChatAPI
from dsrag.reranker import NoReranker, CohereReranker

from dsrag.database.vector.chroma_db import ChromaDB
from dsrag.document_parsing import extract_text_from_pdf

llm = OpenAIChatAPI(model='gpt-4o-mini')
reranker = CohereReranker()

com_id = "Commercial_Aerospace"
uss_id = "USS"

def create_kb(kb_id, folder_path):
    Kb = KnowledgeBase(kb_id, reranker = CohereReranker(), vector_db=ChromaDB(kb_id), storage_directory="~/AI-Agents-For-ST/storage")
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        id = get_file_as_id(file_path)
        print(id)
        text = extract_text_from_pdf(file_path)
        Kb.add_document(doc_id=id, text=text[0])
    return Kb



com_kb = create_kb(com_id, folder_com)
uss_kb = create_kb(uss_id, folder_uss)


# com_kb = KnowledgeBase(com_id, reranker = NoReranker(), vector_db=ChromaDB(kb_id), storage_directory="~/AI-Agents-For-ST/storage")

CommAero_AAM_AerSaleCorp_2022
Adding 511 chunks to the database
USS_Satcom_Comtech_2021
Adding 768 chunks to the database


In [6]:
import pymupdf
from dsrag.document_parsing import extract_text_from_pdf
    
print(extract_text_from_pdf("/home/user/AI-Agents-For-ST/Dataset/Commercial Aero/CommAero_AAM_AirLeaseCorp_2022.pdf")[1][1])

company,” and “emerging growth company” in Rule 12b-2 of the Exchange Act.
Large accelerated filer
☒
Accelerated filer
☐
Non-accelerated filer
☐
Smaller reporting company
☐
Emerging growth company
☐
If an emerging growth company, indicate by check mark if the Registrant has elected not to use the extended transition period for complying with any new or
revised financial accounting standards provided pursuant to Section 13(a) of the Exchange Act. ☐
Indicate by check mark whether the registrant has filed a report on and attestation to its management’s assessment of the effectiveness of its internal control
over financial reporting under Section 404(b) of the Sarbanes-Oxley Act (15 U.S.C. 7262(b)) by the registered public accounting firm that prepared or issued its
audit report. Yes ☒ No ☐
If securities are registered pursuant to Section 12(b) of the Act, indicate by check mark whether the financial statements of the registrant included in the filing
reflect the correction of an error to 

In [6]:
from dsrag.knowledge_base import KnowledgeBase
from dsrag.llm import OpenAIChatAPI
from dsrag.reranker import NoReranker, CohereReranker

from dsrag.database.vector.chroma_db import ChromaDB
from dsrag.database.vector.milvus_db import MilvusDB
from dsrag.document_parsing import extract_text_from_pdf

llm = OpenAIChatAPI(model='gpt-4o-mini')
reranker = CohereReranker()

com_id = "Commercial_Aerospace"
uss_id = "USS"

def Kb_add_doc(Kb, file_path):
    id = get_file_as_id(file_path)
    print(id)
    text = extract_text_from_pdf(file_path)
    Kb.add_document(doc_id=id, text=text[0])
    return Kb

com_kb = KnowledgeBase(com_id, reranker = CohereReranker(), vector_db=ChromaDB(com_id), storage_directory="~/AI-Agents-For-ST/storage")
uss_kb = KnowledgeBase(uss_id, reranker = CohereReranker(), vector_db=ChromaDB(uss_id), storage_directory="~/AI-Agents-For-ST/storage")


# com_kb = KnowledgeBase(com_id, reranker = NoReranker(), vector_db=ChromaDB(kb_id), storage_directory="~/AI-Agents-For-ST/storage")

In [7]:
count_chunks=0
for doc_id in com_kb.chunk_db.data:
    count_chunks += len(com_kb.chunk_db.data[doc_id])
print(count_chunks)

count_chunks=0
for doc_id in uss_kb.chunk_db.data:
    count_chunks += len(uss_kb.chunk_db.data[doc_id])
print(count_chunks)

514
757


In [3]:
import time
from collections import defaultdict
com_kb = KnowledgeBase(com_id, reranker = CohereReranker(), vector_db=ChromaDB(com_id), storage_directory="~/AI-Agents-For-ST/storage")
len_com = len(com_kb.chunk_db.data)

questions = ["Revenue", "Income", "Assets", "Equity", "Liabilities"]
# questions = ["Assets", "Equity", "Liabilities"]
failures_com = defaultdict(list)
not_in_text_com = defaultdict(list)

for q in questions:
    for id in com_file_id:
        stop_value = 0
        year = id[-4:]
        company = id[:-4]
        query = [f"What is the total {q} for {company} in {year}"]
        time.sleep(7)
        try:
            flag = False
            document = com_kb.query(query)
            for i in range(len(document)):
            # assert document == doc_id if not add to failures
                if document[i]["doc_id"] == id:
                    stop_value = i
                    flag = True
                    # print("OK")
                    pass

                else:
                    break
                    
            if not flag:
                print(f"Mismatch id for {id} with {document[i]['doc_id']}")
                failures_com[q].append(id)
                not_in_text_com[q].append(id)
            
            if flag:
                
                flag = False

                for i in range(stop_value):
                    if q in document[i]["text"] or q.lower() in document[i]["text"]:
                        flag = True
                        break
                
                if not flag:
                    print(f"Failed for {q} with not in text returned for {id}")
                    not_in_text_com[q].append(id)


        except Exception as e:
            print(f"Failed for {q} because of {e} for {id}")
            if id not in failures_com[q]:
                failures_com[q].append(id)
            if id not in not_in_text_com[q]:
                not_in_text_com[q].append(id)

    print(f"Failure % for {q} in com_file_id is",len(failures_com[q])/len_com)
    print(f"Not in text % for {q} in com_file_id is",len(not_in_text_com[q])/len_com)





# query = ["What is the revenue of General SpiritAeroSystems 2021"]
# print(com_kb.query(query)[0])
# print(com_kb.query(query)[0]["doc_id"])
# print(len(test.chunk_db.data))

Failed for Revenue with not in text returned for CabinInterior JAMCO 2022
Failed for Revenue with not in text returned for General SpiritAeroSystems 2021
Mismatch id for AAM AerSaleCorp 2022 with AAM AerSaleCorp 2023
Failed for Revenue with not in text returned for CompositePanels ParkAerospace 2022
Failed for Revenue with not in text returned for Generall AARCorp 2022
Failed for Revenue because of list index out of range for CabinInterior DroneDelivery 2021
Mismatch id for CompositePanels Tejin 2022 with CompositePanels Tejin 2023
Failed for Revenue with not in text returned for CompositePanels AirAsiaX 2021
Failed for Revenue with not in text returned for AAM WillisLease 2022
Failed for Revenue because of list index out of range for CabinInterior DroneDelivery 2022
Failed for Revenue with not in text returned for CompositePanels ParkAerospace 2021
Failed for Revenue with not in text returned for CabinInterior JAMCO 2021
Failed for Revenue with not in text returned for EngineServices 

In [6]:
failures_com


defaultdict(list,
            {'Revenue': ['AAM AerSaleCorp 2022',
              'CabinInterior DroneDelivery 2021',
              'CompositePanels Tejin 2022',
              'CabinInterior DroneDelivery 2022',
              'General SIAEngineering 2022'],
             'Income': ['AAM AerSaleCorp 2022',
              'CompositePanels Tejin 2022',
              'CabinInterior DroneDelivery 2022',
              'General SIAEngineering 2022'],
             'Assets': ['AAM AerSaleCorp 2022',
              'CompositePanels Tejin 2022',
              'CabinInterior DroneDelivery 2022',
              'ComponentServices AirFranceKLM 2021',
              'General SIAEngineering 2022'],
             'Equity': ['AAM AerSaleCorp 2022',
              'CompositePanels Tejin 2022',
              'General SIAEngineering 2022'],
             'Liabilities': ['CabinInterior JAMCO 2022',
              'AAM AerSaleCorp 2022',
              'CompositePanels Tejin 2022',
              'AAM WillisLease 2022',

In [7]:
not_in_text_com

defaultdict(list,
            {'Revenue': ['CabinInterior JAMCO 2022',
              'General SpiritAeroSystems 2021',
              'AAM AerSaleCorp 2022',
              'CompositePanels ParkAerospace 2022',
              'Generall AARCorp 2022',
              'CabinInterior DroneDelivery 2021',
              'CompositePanels Tejin 2022',
              'CompositePanels AirAsiaX 2021',
              'AAM WillisLease 2022',
              'CabinInterior DroneDelivery 2022',
              'CompositePanels ParkAerospace 2021',
              'CabinInterior JAMCO 2021',
              'EngineServices Raytheon 2022',
              'General SIAEngineering 2022'],
             'Income': ['AAM AerSaleCorp 2022',
              'Generall AARCorp 2022',
              'CabinInterior DroneDelivery 2021',
              'CompositePanels Tejin 2022',
              'AAM WillisLease 2022',
              'CabinInterior DroneDelivery 2022',
              'General SIAEngineering 2022'],
             'Assets':

In [8]:
questions = ["Revenue", "Income", "Assets", "Equity", "Liabilities"]
len_uss = len(uss_kb.chunk_db.data)
import time
from collections import defaultdict

failures_uss = defaultdict(list)
not_in_text_uss = defaultdict(list)
uss_kb = KnowledgeBase(uss_id, reranker = CohereReranker(), vector_db=ChromaDB(uss_id), storage_directory="~/AI-Agents-For-ST/storage")

for q in questions:
    for id in uss_file_id:
        year = id[-4:]
        company = id[:-4]
        query = [f"What is the total {q} for {company} in {year}"]
        time.sleep(7)
        try:
            flag = False
            document = uss_kb.query(query)
            for i in range(len(document)):
            # assert document == doc_id if not add to failures
                if document[i]["doc_id"] == id:
                    stop_value = i
                    flag = True
                    pass
                else:
                    break
            
            if not flag:
                print(f"Mismatch id for {id} with {document[i]['doc_id']}")
                # print(q)
                failures_uss[q].append(id)
                not_in_text_uss[q].append(id)
                # print(failures_uss)

            if flag:
                flag = False

                for i in range(stop_value):
                    if q in document[i]["text"] or q.lower() in document[i]["text"]:
                        flag = True
                        break
                
                if not flag:
                    print(f"Failed for {q} with not in text returned for {id}")
                    not_in_text_uss[q].append(id)
        except Exception as e:
            print(f"Failed for {q} because of {e}")
            if id not in failures_uss[q]:
                failures_uss[q].append(id)
            if id not in not_in_text_uss[q]:
                not_in_text_uss[q].append(id)

    print(f"Failure % for {q} in uss_file_id is", len(failures_uss[q])/len_uss)
    print(f"Not in text % for {q} in uss_file_id is",len(not_in_text_uss[q])/len_uss)

Failed for Revenue with not in text returned for Satcom Comtech 2021
Failed for Revenue with not in text returned for Satcom Gilat 2021
Mismatch id for Satcom Echostar 2022 with Satcom Echostar 2021
Failure % for Revenue in uss_file_id is 0.125
Not in text % for Revenue in uss_file_id is 0.375
Mismatch id for Satcom Echostar 2022 with Satcom Echostar 2021
Failure % for Income in uss_file_id is 0.125
Not in text % for Income in uss_file_id is 0.125
Mismatch id for Satcom Echostar 2022 with Satcom Echostar 2021
Failure % for Assets in uss_file_id is 0.125
Not in text % for Assets in uss_file_id is 0.125
Mismatch id for Satcom Echostar 2022 with Satcom Echostar 2021
Failure % for Equity in uss_file_id is 0.125
Not in text % for Equity in uss_file_id is 0.125
Mismatch id for Satcom Echostar 2022 with Satcom Echostar 2021
Failure % for Liabilities in uss_file_id is 0.125
Not in text % for Liabilities in uss_file_id is 0.125


In [9]:
failures_uss

defaultdict(list,
            {'Revenue': ['Satcom Echostar 2022'],
             'Income': ['Satcom Echostar 2022'],
             'Assets': ['Satcom Echostar 2022'],
             'Equity': ['Satcom Echostar 2022'],
             'Liabilities': ['Satcom Echostar 2022']})

In [10]:
not_in_text_uss

defaultdict(list,
            {'Revenue': ['Satcom Comtech 2021',
              'Satcom Gilat 2021',
              'Satcom Echostar 2022'],
             'Income': ['Satcom Echostar 2022'],
             'Assets': ['Satcom Echostar 2022'],
             'Equity': ['Satcom Echostar 2022'],
             'Liabilities': ['Satcom Echostar 2022']})

In [9]:
kb = com_kb
kb_id = "Commerical_Aerospace"
doc_id = "CommAero_AAM_AerSaleCorp_2022" 

num_chunks = len(com_kb.chunk_db.data[doc_id])
print (num_chunks)

chunks = []
for i in range(num_chunks):
    chunk = {
        "kb_id": kb_id,
        "doc_id": doc_id,
        "section_title": kb.chunk_db.get_section_title(doc_id, i),
        "section_summary": kb.chunk_db.get_section_summary(doc_id, i),
        "chunk_text": kb.chunk_db.get_chunk_text(doc_id, i),
        "document_title": kb.chunk_db.get_document_title(doc_id, i),
        "document_summary": kb.chunk_db.get_document_summary(doc_id, i)
    }

    chunks.append(chunk)

chunks[1]
    

514


{'kb_id': 'Commerical_Aerospace',
 'doc_id': 'CommAero_AAM_AerSaleCorp_2022',
 'section_title': 'Introduction to the Annual Report and Company Overview',
 'section_summary': '',
 'chunk_text': 'Securities registered pursuant to Section 12(b) of the Act:\nTitle of each class\nTrading Symbol(s)\nName of each exchange on which registered\nCommon Stock, par value $0.0001 per share\nASLE\nThe Nasdaq Global Market\nSecurities registered pursuant to Section 12(g) of the Act: None\nIndicate by check mark if the registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act. Yes \n☐ No ☒\nIndicate by check mark if the registrant is not required to file reports pursuant to Section 13 or Section 15(d) of the Act. Yes ☐ No ☒\nIndicate by check mark whether the registrant: (1) has filed all reports required to be filed by Section 13 or 15(d) of the Securities Exchange Act of 1934 during the preceding 12 months (or for such shorter period that the registrant',
 'document_ti

In [6]:
unique_section_titles = []
for i in range(num_chunks):
    section_title = chunks[i]["section_title"]
    if section_title not in unique_section_titles:
        print (section_title)
        unique_section_titles.append(section_title)

Consolidated Financial Results Overview for Fiscal Year 2022
Dividend Distribution Information
Financial Results Forecast for Fiscal Year 2023
Non-Consolidated Financial Results Overview for Fiscal Year 2022
Overview of Business Results and Financial Position
Future Outlook for the Air Transportation Industry
Document Notes and Overview
Table of Contents
Overview of Business Results for the Fiscal Year Under Review
Overview of Financial Position for the Fiscal Year Under Review
Overview of Cash Flows for the Fiscal Year Under Review
Future Outlook
Basic Approach to the Selection of Accounting Standards
Consolidated Financial Statements Overview
Consolidated Balance Sheets
Consolidated Statements of Income and Comprehensive Income
Consolidated Statements of Changes in Equity
Consolidated Statements of Cash Flows
Notes to Consolidated Financial Statements


In [32]:
# print(not_in_text_com)
# print(not_in_text_uss)
# com_kb = KnowledgeBase(com_id, reranker = CohereReranker(), vector_db=MilvusDB(com_id), storage_directory="~/AI-Agents-For-ST/storage")
import time

#measure time taken to query

start = time.time()
com_kb = KnowledgeBase(com_id, reranker = CohereReranker(), vector_db=ChromaDB(com_id), storage_directory="~/AI-Agents-For-ST/storage")
query = [f"What is the revenue of {max_chunk_id}"]
print(com_kb.query(query))
end = time.time()
time_taken = end - start
print("Time taken for chroma db is", time_taken)
start = time.time()
com_kb = KnowledgeBase(com_id, reranker = CohereReranker(), vector_db=MilvusDB(com_id), storage_directory="~/AI-Agents-For-ST/storage")
query = [f"What is the revenue of {max_chunk_id}"]
print(com_kb.query(query))
end = time.time()
time_taken = end - start
print("Time taken for milvus db is", time_taken)

print(uss_kb.vector_db)

[{'doc_id': 'ComponentServices AirFranceKLM 2022', 'chunk_start': 3085, 'chunk_end': 3094, 'score': 1.1898191857388636, 'text': "Document context: the following excerpt is from a document titled 'UNIVERSAL REGISTRATION DOCUMENT 2022 INCLUDING THE ANNUAL FINANCIAL REPORT'. This document is about: the 2022 Universal Registration Document for Air France-KLM, detailing the group's financial performance, corporate governance, risk management, and sustainability initiatives.\n\n€333\xa0million and €283\xa0million for the periods ended December\xa031, \n2022 and December\xa031, 2021.\nAmsterdam Airport Schiphol (AAS)\n > land and property rental agreements; \n > airport and passenger -related fee arrangements.\nIn addition, AAS collects airport fees on behalf of the Dutch State.\nTotal expenses incurred by the Group in connection with the \nafore -mentioned arrangements amounted to €71\xa0million for \nthe period ended December\xa031, 2022 versus €76\xa0million as of \nDecember\xa031, 2021.\n

In [10]:
from dsrag.knowledge_base import KnowledgeBase
from dsrag.llm import OpenAIChatAPI
from dsrag.reranker import NoReranker, CohereReranker

from dsrag.database.vector.chroma_db import ChromaDB
from dsrag.document_parsing import extract_text_from_pdf


llm = OpenAIChatAPI(model='gpt-4o-mini')
reranker = CohereReranker()

com_id = "Commercial_Aerospace"
uss_id = "USS"

com_kb = KnowledgeBase(com_id, reranker = CohereReranker(), vector_db=ChromaDB(com_id), storage_directory="~/AI-Agents-For-ST/storage")
uss_kb = KnowledgeBase(uss_id, reranker = CohereReranker(), vector_db=ChromaDB(uss_id), storage_directory="~/AI-Agents-For-ST/storage")



system = {"role":"system", "content":"You are a financial analyst at a large investment firm. You have been asked to analyze the financial performance of a company."}

question = "What is the revenue of CommAero_AAM_AerSaleCorp_2022"

def get_context(kb, query):
    document = kb.query(query)
    context = document[0]["text"]
    return context

user = """

{context}

Based on the information provided, answer the following questions:

{question}
"""

query = [question]
context = get_context(com_kb, query)

# print(context)

prompt = user.format(context=context, question=question)

prompt = {"role":"user", "content":prompt}

messages = [system, prompt]

llm = OpenAIChatAPI(model='gpt-4o-mini')


message = []
response = llm.make_llm_call(messages)
print(response)



Document context: the following excerpt is from a document titled 'FORM 10-K'. This document is about: the annual financial performance and operational overview of AerSale Corporation for the fiscal year ended December 31, 2021, as reported in their Form 10-K filing with the SEC.

Results of Operations
Sales and gross profit for AerSale’s two business segments for the years ended in December 31, 2021 and 2020 were as follows:
Year ended December 31, 2021 compared to the year ended December 31, 2020
Year Ended December 31, 
 
(in thousands, except percentages)
    
2021
    
2020
    
Percent Change
 
Revenue
  
  
  
 
Asset Management Solutions
 
   
   
  
Aircraft
$
 87,461
$
 53,639  
 63.1 %
Engines
 
 144,549
 
 45,072  
 220.7 %
$
 232,010
$
 98,711  
 135.0 %
TechOps
 
  
 
   
  
MRO
$
 99,899
$
 103,899  
 (3.8)%
Product Sales
 
 8,528
 
 6,328  
 34.8 %
$
 108,427
$
 110,227  
 (1.6)%
Total
$
 340,437
$
 208,938  
 62.9 %
Table of Contents
36
Year Ended December 31, 
 
(in t